In [1]:
#imports
import pandas as pd
import numpy as np
import sklearn
import ssl
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
train_url = "https://ml.monov.eu/coffee/train_data.csv"
test_url = "https://ml.monov.eu/coffee/test_data.csv"


In [2]:

train_data = pd.read_csv(train_url)
test_data = pd.read_csv(test_url)

train_data.isnull().sum()
#
train_data.dropna(inplace=True)
# turning Altitude from string to numeric
train_data['Altitude'] = pd.to_numeric(train_data['Altitude'], errors='coerce')
train_data['Altitude'].fillna(train_data['Altitude'].mean(), inplace=True)
# fill altitude
train_data['Altitude'].fillna(train_data['Altitude'].mean(), inplace=True)

# fix for harvest
train_data['Harvest.Year.Start'] = train_data['Harvest.Year'].str.split(' / ').str[0]
train_data['Harvest.Year.End'] = train_data['Harvest.Year'].str.split(' / ').str[1]
train_data['Harvest.Year.Start'] = pd.to_numeric(train_data['Harvest.Year.Start'], errors='coerce')
train_data['Harvest.Year.End'] = pd.to_numeric(train_data['Harvest.Year.End'], errors='coerce')
# fill the missing data
train_data['Harvest.Year.Start'].fillna(train_data['Harvest.Year.Start'].median(), inplace=True)
train_data['Harvest.Year.End'].fillna(train_data['Harvest.Year.End'].median(), inplace=True)
train_data['Farm.Name'].fillna('Unknown', inplace=True)
train_data['Mill'].fillna('Unknown', inplace=True)
train_data['Owner'].fillna(train_data['Owner'].mode()[0], inplace=True)
train_data['Processing.Method'].fillna(train_data['Processing.Method'].mode()[0], inplace=True)
train_data.isnull().sum()



Id                       0
Species                  0
Owner                    0
Country.of.Origin        0
Farm.Name                0
Lot.Number               0
Mill                     0
ICO.Number               0
Company                  0
Altitude                 0
Region                   0
Producer                 0
Number.of.Bags           0
Bag.Weight               0
In.Country.Partner       0
Harvest.Year             0
Grading.Date             0
Owner.1                  0
Variety                  0
Processing.Method        0
Coffee.Quality           0
Moisture                 0
Category.One.Defects     0
Quakers                  0
Color                    0
Category.Two.Defects     0
Expiration               0
Certification.Body       0
Certification.Address    0
Certification.Contact    0
unit_of_measurement      0
altitude_low_meters      0
altitude_high_meters     0
altitude_mean_meters     0
Harvest.Year.Start       0
Harvest.Year.End         0
dtype: int64

In [3]:
test_data.isnull().sum()
test_data.dropna(inplace=True)
test_data['Altitude'].fillna(test_data['Altitude'].mean(), inplace=True)
test_data['Region'].replace({'InconsistentValue': 'CorrectValue'}, inplace=True)
test_data.isnull().sum()


Id                       0
Species                  0
Owner                    0
Country.of.Origin        0
Farm.Name                0
Lot.Number               0
Mill                     0
ICO.Number               0
Company                  0
Altitude                 0
Region                   0
Producer                 0
Number.of.Bags           0
Bag.Weight               0
In.Country.Partner       0
Harvest.Year             0
Grading.Date             0
Owner.1                  0
Variety                  0
Processing.Method        0
Moisture                 0
Category.One.Defects     0
Quakers                  0
Color                    0
Category.Two.Defects     0
Expiration               0
Certification.Body       0
Certification.Address    0
Certification.Contact    0
unit_of_measurement      0
altitude_low_meters      0
altitude_high_meters     0
altitude_mean_meters     0
dtype: int64

In [4]:
columnsToDrop = ["Id","Lot.Number","ICO.Number","Number.of.Bags","Bag.Weight","In.Country.Partner","Grading.Date","Owner.1","Quakers","Expiration","Certification.Address","Certification.Contact","unit_of_measurement"]
train_data.drop(columnsToDrop, axis=1, inplace=True)
test_data.drop(columnsToDrop, axis=1, inplace=True)

In [5]:
unnecessary_features = ['altitude_low_meters', 'altitude_high_meters', 'altitude_mean_meters']
unnecessary_features2 = ['Harvest.Year.End', 'Harvest.Year.Start']
train_data.drop(columns=unnecessary_features, inplace=True)
train_data.drop(columns=unnecessary_features2, inplace=True)
test_data.drop(columns=unnecessary_features, inplace=True)
x_train = train_data.drop(columns=['Coffee.Quality'])
y_train = train_data['Coffee.Quality']

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numeric_features = x_train.select_dtypes(include=['int64', 'float64']).columns
x_train[numeric_features] = scaler.fit_transform(x_train[numeric_features])


In [7]:
non_numeric_features = x_train.dtypes[x_train.dtypes == 'object'].index
encoder = LabelEncoder()
for feature in non_numeric_features:
    x_train[feature] = encoder.fit_transform(x_train[feature])
for feature in non_numeric_features:
    test_data[feature] = encoder.fit_transform(test_data[feature])
x = x_train
y = y_train
x_train = x_train.fillna(0)
test_data = test_data.fillna(0)
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.20, random_state=42)


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

models = {
    "Logistic Regression": LogisticRegression(),
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Boosting Classifier": GradientBoostingClassifier()
}




In [9]:
results = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[model_name] = accuracy



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
results_df = pd.DataFrame.from_dict(results, orient="index", columns=["Accuracy"])
best_model_name = max(results, key=results.get)
best_model = models[best_model_name]
best_model.fit(x, y)
test_predictions = best_model.predict(test_data)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
results_dict = {"Model": list(results.keys()), "Accuracy": list(results.values())}
results_dict["Coffee.Quality"] = test_predictions
import csv
with open("result.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=results_dict.keys())
    writer.writeheader()
    writer.writerow(results_dict)